In [1]:
import pImpactR as impact
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as copy
import pickle

rectangular_ = 1
elliptic_ = 2

nCore_y = 2
nCore_z = 4
npt = 50000
nturn = 2000

In [2]:
emitGeomRMS = 3.3e-6
dE = 2.0e-3

In [3]:
NL_t = 0.4
NL_L = 1.8
NL_c = 0.01
NL_nu = 0.3

# read Aperture

In [4]:
beam,lattice = impact.readInputFile('../test.Chad.in')
ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency
g = ke/mass+1.0
bg = np.sqrt(g**2-1.0)
emitN = emitGeomRMS*bg

reading ImpactZ input file (../test.Chad.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [5]:
def getLostDataThermal(deco,H0,tauMag,tauBeam,npt,nturn,order=3):
    
    # read file
    beam,lattice = impact.readInputFile('test_'+deco+'.in')
    beam.nCore_y=nCore_y
    beam.nCore_z=nCore_z
    beam.n_particles = npt
    ke = beam.kinetic_energy
    mass = beam.mass
    freq = beam.frequency
    
    
    # adjust kick number
    cleanLat = impact.clearLattice(lattice)
    for item in cleanLat:
        if item.type == 'RFkick':
            item.vmax = 0.0
        if item.type == 'nonlinear_insert':
            NLfoward = item
        if 'length' in item:
            item.n_sckick = int(np.ceil(item.length*50))
            item.n_map = 1
    NLfoward.n_sckick = 2
    NLfoward.n_map = 45
    NLfoward.strength_t = tauMag
    
    
    # add QFF
    cleanLat = impact.addHardEdgeQuad(cleanLat)
    
    
    # ==== run backward (to injection point) =====
    for i in range(len(cleanLat)):
        if cleanLat[i].type == 'nonlinear_insert':
            break

    latticeB = impact.getInverseLattice(cleanLat[:i])
    
    write0 = impact.getElem('write_raw_ptcl')
    write0.file_id = 111110
    write0.format_id = 2
    latticeB.insert(0,write0)

    write1 = impact.getElem('write_raw_ptcl')
    write1.file_id = 111111
    write1.format_id = 2
    latticeB.append(write1)

    loop = impact.getElem('loop')
    loop.turns = 1
    latticeB.insert(0,loop)
    
    beam.distribution.mode = 'twiss'
    beam.distribution.distribution_type = 'IOTA_Gauss'
    beam.distribution.NL_t  = tauBeam
    beam.distribution.NL_c  = NL_c
    beam.distribution.betx  = NL_L/np.sin(2.0*np.pi*NL_nu)
    beam.distribution.alfx  = np.tan(np.pi*NL_nu)
    beam.distribution.emitx = H0
    beam.distribution.CL    = 6.0
    beam.distribution.betz  = 200
    beam.distribution.alfz  = 0.0
    beam.distribution.emitz = (dE*ke*1.0e-6)**2*beam.distribution.betz
    
    impact.writeInputFile(beam,latticeB)
    impact.run(beam,order=order)
    
    pData0 = impact.readParticleData(111110,ke,mass,freq,2)
    Twiss0 = impact.getTwiss_from_pData(pData0)
    pData1 = impact.readParticleData(111111,ke,mass,freq,2)
    Twiss1 = impact.getTwiss_from_pData(pData1)
    
    
    # ==== run foward (from injection point) ====
    cleanLat.insert(0,impact.getElem('pipeinfo'))
    cleanLat.insert(1,impact.getElem('loop'))
    cleanLat[1].turns = nturn
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = 111111
    impact.writeInputFile(beam,cleanLat)
    impact.run(beam,order=order)
    
    
    # ==== get Data ====
    try:
        pLost = np.loadtxt('lost_partcl.data',skiprows=1, usecols=(0,1,2,3))
    except:
        pLost = None
    
    fort81 = np.loadtxt('fort.81',usecols=(0,1,2,3,4,5))
    Hmean = fort81[:,1]
    Imean = fort81[:,2]
    Hstd  = fort81[:,3]
    Istd  = fort81[:,4]
    HIcor = fort81[:,5]
    
    emitx = np.loadtxt('fort.24',usecols=(0, -1))
    s = emitx[:,0]
    emitx = emitx[:,-1]
    emity = np.loadtxt('fort.25',usecols=(-1))
    
    stat = {'s'    :s,
            'emitx':emitx,
            'emity':emity,
            'Hmean':Hmean,
            'Imean':Imean,
            'Hstd' :Hstd,
            'Istd' :Istd,
            'HIcor':HIcor}
    
    data = {'Twiss0':Twiss0,
            'Twiss1':Twiss1,
            'pData0':pData0,
            'pData1':pData1,
            'pLost' :pLost,
            'stat'  :stat}

    #!rm lost_partcl.data fort.* test.in
    
    return data

In [24]:
def getLostDataExponential(deco,betx,alfx,emitx,bety,alfy,emity,tauMag,npt,nturn,order=3):
    
    # read file
    beam,lattice = impact.readInputFile('test_'+deco+'.in')
    beam.nCore_y=nCore_y
    beam.nCore_z=nCore_z
    beam.n_particles = npt
    
    beam.distribution.mode = 'twiss'
    beam.distribution.distribution_type = 'Exponential2D_trunc'
    beam.distribution.betx  = betx
    beam.distribution.alfx  = alfx
    beam.distribution.emitx = emitx
    beam.distribution.CLx   = 6.0
    beam.distribution.bety  = bety
    beam.distribution.alfy  = alfy
    beam.distribution.emity = emity
    beam.distribution.CLy   = 6.0
    beam.distribution.betz  = 200
    beam.distribution.alfz  = 0.0
    beam.distribution.emitz = (dE*ke*1.0e-6)**2*beam.distribution.betz
    
    # adjust kick number
    cleanLat = impact.clearLattice(lattice)
    for item in cleanLat:
        if item.type == 'RFkick':
            item.vmax = 0.0
        if item.type == 'nonlinear_insert':
            NLfoward = item
        if 'length' in item:
            item.n_sckick = int(np.ceil(item.length*50))
            item.n_map = 1
    NLfoward.n_sckick = 2
    NLfoward.n_map = 45
    NLfoward.strength_t = tauMag
    
    
    # add QFF
    cleanLat = impact.addHardEdgeQuad(cleanLat)
    
    # ==== run foward (from injection point) ====
    cleanLat.insert(0,impact.getElem('pipeinfo'))
    cleanLat.insert(1,impact.getElem('loop'))
    cleanLat[1].turns = nturn
    
    write1 = impact.getElem('write_raw_ptcl')
    write1.file_id = 111111
    write1.format_id = 2
    
    impact.writeInputFile(beam,cleanLat[:2]+[write1]+cleanLat[2:])
    impact.run(beam,order=order)
    pData1 = impact.readParticleData(111111,ke,mass,freq,2)
    Twiss1 = impact.getTwiss_from_pData(pData1)
    
    try:
        pLost = np.loadtxt('lost_partcl.data',skiprows=1, usecols=(0,1,2,3))
    except:
        pLost = None
        
    
    # ==== get Data ====
    fort81 = np.loadtxt('fort.81',usecols=(0,1,2,3,4,5))
    Hmean = fort81[:,1]
    Imean = fort81[:,2]
    Hstd  = fort81[:,3]
    Istd  = fort81[:,4]
    HIcor = fort81[:,5]
    
    emitx = np.loadtxt('fort.24',usecols=(0, -1))
    s = emitx[:,0]
    emitx = emitx[:,-1]
    emity = np.loadtxt('fort.25',usecols=(-1))
    
    stat = {'s'    :s,
            'emitx':emitx,
            'emity':emity,
            'Hmean':Hmean,
            'Imean':Imean,
            'Hstd' :Hstd,
            'Istd' :Istd,
            'HIcor':HIcor}
    
    data = {'Twiss0':Twiss0,
            'Twiss1':Twiss1,
            'pData0':pData0,
            'pData1':pData1,
            'pLost' :pLost,
            'stat'  :stat}
    
    #!rm lost_partcl.data fort.111111 test.in
    
    return data

# Run and collect data

In [7]:
Deco = ['iota_v8_4_SextOff_NLon',
        'iota_v8_4_SextOn_NLon',
        'iota_v8_4_SextGoodDA_NLon']
data = pickle.load(open('pLossData_iota_v8_4_variousSextSettings.pickle','rb'))
# data = {}

## Thermal Beam

#### iota_v8_4_SextOff_NLon

In [ ]:
deco = 'iota_v8_4_SextOff_NLon'
if deco+'_thermal' not in data.keys():
    data[deco+'_thermal']=getLostDataThermal(deco,emitGeomRMS,0.4,0.4,npt,nturn)
    pickle.dump(data,open('pLossData_iota_v8_4_variousSextSettings.pickle','wb'))

reading ImpactZ input file (test_iota_v8_4_SextOff_NLon.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


#### iota_v8_4_SextOn_NLon

In [ ]:
deco = 'iota_v8_4_SextOn_NLon'
if deco+'_thermal' not in data.keys():
    data[deco+'_thermal']=getLostDataThermal(deco,emitGeomRMS,0.4,0.4,npt,nturn)
    pickle.dump(data,open('pLossData_iota_v8_4_variousSextSettings.pickle','wb'))

#### iota_v8_4_SextGoodDA_NLon

In [ ]:
deco = 'iota_v8_4_SextGoodDA_NLon'
if deco+'_thermal' not in data.keys():
    data[deco+'_thermal']=getLostDataThermal(deco,emitGeomRMS,0.4,0.4,npt,nturn)
    pickle.dump(data,open('pLossData_iota_v8_4_variousSextSettings.pickle','wb'))

## Exponential Beam

In [22]:
deco = 'iota_v8_4_SextOff_NLon'
Twiss1 = data[deco+'_thermal']['Twiss1']
betx,alfx,emitx,bety,alfy,emity,betz,alfz,emitz = Twiss1
Twiss1

(0.49524020293287413,
 0.16669260330749974,
 2.645169074698375e-06,
 2.047576820369176,
 0.4621109662791848,
 5.5814862683766e-06,
 1338.7005768170793,
 2.3611888931473417,
 0.0051337593474649855)

#### iota_v8_4_SextOff_NLon

In [ ]:
deco = 'iota_v8_4_SextOff_NLon'
if deco+'_exponential' not in data.keys():
    data[deco+'_exponential']=getLostDataExponential(deco,betx,alfx,emitx,bety,alfy,emity,0.4,npt,nturn,order=3)
    pickle.dump(data,open('pLossData_iota_v8_4_variousSextSettings.pickle','wb'))

reading ImpactZ input file (test_iota_v8_4_SextOff_NLon.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


#### iota_v8_4_SextOn_NLon

In [ ]:
deco = 'iota_v8_4_SextOn_NLon'
if deco+'_exponential' not in data.keys():
    data[deco+'_exponential']=getLostDataExponential(deco,betx,alfx,emitx,bety,alfy,emity,0.4,npt,nturn,order=3)
    pickle.dump(data,open('pLossData_iota_v8_4_variousSextSettings.pickle','wb'))

#### iota_v8_4_SextGoodDA_NLon

In [ ]:
deco = 'iota_v8_4_SextGoodDA_NLon'
if deco+'_exponential' not in data.keys():
    data[deco+'_exponential']=getLostDataExponential(deco,betx,alfx,emitx,bety,alfy,emity,0.4,npt,nturn,order=3)
    pickle.dump(data,open('pLossData_iota_v8_4_variousSextSettings.pickle','wb'))